In [1]:
import pandas as pd
import numpy as np
import pandasql as ps

#libraries for web scrapping
import requests
from bs4 import BeautifulSoup
import urllib.request
import time
import random

#libraries to connect to API
import json

In [43]:
#function to load API Key locally stored
def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [24]:
#Import reference data to the notebook

#This is the list of counties for the study
county_selection = pd.read_csv("MENUISM3/county_selection.csv")

#This document will be used in several modules through the notebook
allzipUSA = pd.read_csv("MENUISM3/zip_state_county.csv") #import document with state/county/zipcode info
allzipUSA['zipcode'] = allzipUSA['zipcode'].astype(str) #zipcode as string type
allzipUSA['zipcode'] = allzipUSA['zipcode'].apply(lambda x: '{0:0>5}'.format(x)) #add leading zeros to len 4 zipcodes

# McDonald's Data Gathering

This notebook has the data gathering process of the number of McDonalds in a sample of counties. Below the general steps followed during this notebook:

1. Select a 827 counties for the study (based on ASC county list)
2. Find the zipcodes corresponding to those 827 counties
3. Use Web Scrapping to find the addresses of the near by McDonald's for each zipcode 
4. Use Google Places API to get the correct zipcode and to confirm if the restaurant is still operating of each restaurant retrieved by the web scrapping process 


In [3]:
#get zipcodes of selected counties for the study
McDonaldsZipCodes = county_selection.merge(allzipUSA, how='left', on=['state_abbrev', 'state_name','county'])

## Web Scrapping

Web Scrapping will be used to get the "near by" McDonald's restaurants by zipcode. The zipcodes to search correspond to the ones from the selected counties for this study.

This web scrapping process does not return the zipcode or county corresponding to the McDonald's address. A second step using Google Maps API will help me confirm zipcode and county. 

**Menuism Website:** https://www.menuism.com/

In [5]:
#create a list of zipcodes to the web scrapping routine
web_scrapping_list = list(McDonaldsZipCodes['zipcode'])
len(web_scrapping_list)

24107

Web scrapping tips:
- Use different user-agents for each web scrapping round
- Delete cookies after every web scrapping round
- Re-start the kernel/open a new jupyter notebook session after every web scrapping round

In [ ]:
#     user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/534.55.3 (KHTML, like Gecko) Version/5.1.3 Safari/534.53.10"
#     user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36'
#     user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0'

In [8]:
#This is a routine for web scraping McDonald's restarurants by zip code 
#Web page: Menuism.com
#Python libraries: urllib.request, beautiful soup and pandas

scrapping_list = web_scrapping_list[0:5] #divide full list into smaller ranges
alldata = pd.DataFrame(columns=['name', 'address', 'city', 'zipcode']) #initialize dataframe
for j, zipi in enumerate(scrapping_list):
    url = "https://www.menuism.com/search?q=McDonald%27s&l=" + zipi + "&p=0&s=match&pp=25&f=0"
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.110 Safari/537.36'
    request = urllib.request.Request(url,headers={'User-Agent': user_agent})
    page = urllib.request.urlopen(request) #load the web page
    soup = BeautifulSoup(page, 'html.parser') #parse HTML document
    addresses = soup.find_all("em") #HTML elements & attributes
    cities = soup.find_all("a", {"onclick":"_gaq.push(['_trackEvent', 'RlRow', 'Click', 'City', null, false]);"})
    names = soup.find_all("a", {"onclick":"_gaq.push(['_trackEvent', 'RlRow', 'Click', 'RL Name', null, false]);"})
    if len(cities) > 0:  #if no results retrieved from web page, skip to else
        lsaddress, lscities, lszipcode, lsnames = [], [], [], []
        for i in range(len(addresses)):
            lsaddress.append(addresses[i].text) 
        lsaddress.remove(lsaddress[0]) #this is specifically for MENUISM web scrapping. Found after inspection 1st item is garbage
        for i in range(len(cities)): #cities and names always have the same length
            lscities.append(cities[i].text)
            lsnames.append(names[i].text)
            lszipcode.append(zipi)
        dctionary = {'name':lsnames,'address':lsaddress, 'city':lscities, 'zipcode': lszipcode} #store data to dictionary
        alldata = pd.concat([alldata,pd.DataFrame(dctionary)], ignore_index=True, sort=False) #transfer data to pandas dataframe
    else: 
        dctionary = {'name':['No_result'],'address':['No_result'], 'city':['No_result'], 'zipcode':[zipi]}  #store collected data to dictionary
        alldata = pd.concat([alldata,pd.DataFrame(dctionary)], ignore_index=True, sort=False)
    print(j, " " , zipi) #for visual control
    time.sleep(random.randint(1, 6))  #sleep to ensure IP address is not blocked
alldata.to_csv('Web_scraping_McDonalds.csv') #save web scrapping data to csv file

0   36507
1   36511
2   36526
3   36527
4   36530


In [50]:
len(addresses)

1

In [ ]:
#save web scrapping data to csv file
alldata.to_csv('MENUISM3/3500_6808MENUISM.csv')

## Google Places API

Google Places API to confirm address from web scrapping and check if the restaurant is still operating. 

Google Places API: https://console.cloud.google.com/google/maps-apis/overview?project=peak-apparatus-272919


**Data Cleanning prior to API requests**

In [10]:
#import web scrapping data
MENUISM1 = pd.read_csv("MENUISM3/0_3500MENUISM.csv")
MENUISM2 = pd.read_csv("MENUISM3/3500_6808MENUISM.csv")

MENUISM = pd.concat([MENUISM1, MENUISM2])

MENUISM = MENUISM.loc[MENUISM['name'] != 'No_result']

#convert zipcode column to str type
MENUISM['zipcode'] = MENUISM['zipcode'].astype(str)
#add leading zeros to zipcode
MENUISM['zipcode'] = MENUISM['zipcode'].apply(lambda x: '{0:0>5}'.format(x))

# Find a duplicate rows from web scrapped data
# Find duplicates on address & city (remember that I put the web scrapping zipcode, might not be the actual zipcode)
# so I could have multiple duplicate addresses returned by different zipcodes due to the nearby reults
duplicateDFRow = MENUISM[MENUISM.duplicated(['address', 'city'], keep='first')]
#create a dataframe with unique McDonalds addresses to later confirm with Google places API
keys = ['address','city', 'zipcode']
i1 = MENUISM.set_index(keys).index
i2 = duplicateDFRow.set_index(keys).index
MENUISM_unique = MENUISM[~i1.isin(i2)] #this is the dataframe to use for Google API

#Split city column by comma
city = MENUISM_unique['city'].str.split(', ', expand=True)
MENUISM_unique['city'] = city[0] 
MENUISM_unique['state'] = city[1] 

MENUISM_unique.drop(labels=['Unnamed: 0','Unnamed: 0.1'], inplace=True, axis=1)

C:\Users\franc\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\franc\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\franc\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.py

In [11]:
#this is the dataframe to use for the Google API requests
MENUISM_unique

,name,address,city,zipcode,state
0,McDonald's,1901 Quintard Ave,Anniston,36201,AL
1,McDonald's,3424 Greenbrier Dear Rd,Anniston,36201,AL
2,McDonald's,5550 Mcclellan Blvd,Anniston,36201,AL
6,McDonald's,1015 Quintard Dr,Oxford,36203,AL
7,McDonald's,92 Plaza Ln,Oxford,36203,AL
...,...,...,...,...,...
17313,McDonald's,11961 Asheville Hwy,Inman,29348,SC
17344,McDonald's,308 W Wade Hampton Blvd,Greer,29652,SC
17345,McDonald's,103 Hammett Bridge Rd,Greer,29652,SC
17346,McDonald's,2137 Old Spartanburg Rd,Greer,29652,SC


**Google API Requests**

In [15]:
#import API Key from local directory
#API keys are sensitive and should not be hard coded to your notebook
keys = get_keys("/Users/franc/.secret/GoogleAPI2.json")
api_key = keys['api_key'] 

In [11]:
#create lists for loop
addres_list = list(MENUISM_unique['address'])
state_list = list(MENUISM_unique['state'])

In [ ]:
len(state_list)

In [ ]:
#Using Google API: Places API: findplacefromtext
placesAPI_data = pd.DataFrame(columns=['formatted_address', 'name', 'permanently_closed'])
for i in range(len(addres_list)):
    address = addres_list[i].replace(' ', '%20') #make sure there are no blank spaces for the URL
    state = state_list[i]
    address_search = "McDonald's%20"+ address + ",%20" + state + ",%20USA"
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input="+ address_search + \
        "&inputtype=textquery&fields=name,formatted_address,permanently_closed&key="+ api_key 
    response = requests.get(url).json()
    placesAPI_data = pd.concat([placesAPI_data, pd.DataFrame(response['candidates'])], ignore_index=True, sort=False) #append retrieved information to a dataframe
    time.sleep(1.25)
    print(i, " " , address_search) #print for visual control

In [ ]:
placesAPI_data.to_csv('MENUISM3/GooglePlacesAPIOutput.csv')

## Data Cleanning

In [ ]:
placesAPI_data = pd.read_csv("MENUISM3/GooglePlacesAPIOutput.csv")

# #Split address column by comma, add name and perm_closed colums and rename split columns
McDonaldsdata = placesAPI_data['formatted_address'].str.split(', ', expand=True)
McDonaldsdata['name'] = placesAPI_data['name'] 
McDonaldsdata['permanently_closed'] = placesAPI_data['permanently_closed'] 
McDonaldsdata.rename(columns={0: "address", 1: "city", 2: "zipcode", 3: "country"}, inplace=True)

#drop duplicate rows
McDonaldsdata.drop_duplicates(['address', 'zipcode'],keep='first',inplace=True) 

#delete rows with permanently closed = True
rowsdrop = McDonaldsdata.loc[McDonaldsdata['permanently_closed'] == True]
McDonaldsdata = McDonaldsdata.drop(rowsdrop.index)

#Drop rows with names different than the ones in the list
rowsdropname = McDonaldsdata.loc[~McDonaldsdata['name'].isin(["McDonald's", "McDonalds", "Walmart Supercenter", "Walmart", "McDonald’s"])]
McDonaldsdata = McDonaldsdata.drop(rowsdropname.index)

#Arrange the info in the correct columns for rows with country == Estados Unidos in column 4
McDonaldsdata.loc[McDonaldsdata[4] == 'Estados Unidos','address'] = McDonaldsdata['address'] + ', ' + McDonaldsdata['city']
McDonaldsdata.loc[McDonaldsdata[4] == 'Estados Unidos','city'] = McDonaldsdata['zipcode']
McDonaldsdata.loc[McDonaldsdata[4] == 'Estados Unidos','zipcode'] = McDonaldsdata['country']
McDonaldsdata.loc[McDonaldsdata[4] == 'Estados Unidos','country'] = McDonaldsdata[4]

#Split zipcode into zipcode and state code
McDonaldsdata[['state_abbrev', 'zipcode', 'extra']] = McDonaldsdata['zipcode'].str.split(" ",expand=True)

#Drop rows with names different than the ones in the list
rowsdropcountry = McDonaldsdata.loc[~McDonaldsdata['country'].isin(["Estados Unidos"])]
McDonaldsdata = McDonaldsdata.drop(rowsdropcountry.index)

McDonaldsdata.drop(labels=[4,'permanently_closed', 'extra'], inplace=True, axis=1)

#count number of McDonalds by zipcode
McDonaldsdataFinal = McDonaldsdata.groupby(['state_abbrev', 'zipcode'], as_index=False)['address'].count()
McDonaldsdataFinal.rename(columns={'address': "McDonaldsCount"}, inplace=True)

#add county to each row based on zipcode. Get zipcode from external file
McDonaldsdataFinal = McDonaldsdataFinal.merge(allzipUSA, how='left', on=['state_abbrev','zipcode'])

McDonaldsdataFinal = McDonaldsdataFinal.groupby(['state_abbrev', 'county'], as_index=False)['McDonaldsCount'].sum()

#exclude the counties that were not in the search list. This because it is likely that the data from
#counties that were not in the original list is a result of the "nearby" restaurants and results are incomplete 
#get the working dataset with zipcodes for the web scrapping process
keys = ['state_abbrev','county']
i1 = McDonaldsZipCodes.set_index(keys).index
i2 = McDonaldsdataFinal.set_index(keys).index
McDonaldsdataFinal = McDonaldsdataFinal[i2.isin(i1)]

In [ ]:
#this is the dataset to use in the study
McDonaldsdataFinal.to_csv('MENUISM3/COMPLETE238McDonaldsCounties.csv')

# Temperature Data Gathering

## Web Scrapping

In [5]:
#Import reference data to the notebook
#This is the list of counties for the study
county_selection = pd.read_csv("MENUISM3/county_selection.csv")

In [6]:
#create lists for loop
county_name = list(county_selection['county'])
state_name = list(county_selection['state_name'])
len(state_name)

824

In [7]:
webscr_county = county_name[787:823]
webscr_state = state_name[787:823]

In [ ]:
#     user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/534.55.3 (KHTML, like Gecko) Version/5.1.3 Safari/534.53.10"
#     user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36'
#     user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0'
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.110 Safari/537.36'

In [48]:
randomtimes = [0.5, 0.75] #these are seconds for the sleep method
df = pd.DataFrame(columns=['county', 'state_name', 'month', 'high/low', 'rain']) #initialize dataframe
for j,zipi in enumerate(webscr_county):
    county = webscr_county[j].replace(' ', '%20')
    state = webscr_state[j] .replace(' ', '%20')
    url = "https://www.google.com/search?q=Average+monthly+temperature+" + county + "+county+" + state
    user_agent =  'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0'
    request = urllib.request.Request(url,headers={'User-Agent': user_agent})
    page = urllib.request.urlopen(request)
    soup = BeautifulSoup(page, 'html.parser')
    month = soup.find_all("span", {"role":"gridcell"})
    print(j)

    i = 0
    while i < len(month):
        txtmonth = month[i].text
        txttemp = month[i+1].text
        txtrain = month[i+2].text
        dctionary = {'county': [webscr_county[j]],'state_name': [webscr_state[j]],'month':[txtmonth],'high/low':[txttemp], 'rain':[txtrain]}
        df = pd.concat([df,pd.DataFrame(dctionary)], ignore_index=True, sort=False)
        i = i + 3

0
1
2


In [23]:
df_final = df.copy()

In [ ]:
df.to_csv('MENUISM3/temperaturedata.csv')

## Data Cleanning

In [117]:
#import web scrapped data
temperature_data = pd.read_csv("MENUISM3/Temperature_rain_data.csv")
temperature_data.drop(labels=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1) #drop unnecessary columns
temperature_data.drop_duplicates(keep='first', inplace=True) #drop duplicate rows

In [110]:
temperature_data

,county,state_name,month,high/low,rain
0,Baldwin,Alabama,mayo,29° / 17°,6 días
1,Baldwin,Alabama,junio,32° / 21°,9 días
2,Baldwin,Alabama,julio,33° / 22°,11 días
3,Baldwin,Alabama,agosto,32° / 22°,11 días
4,Baldwin,Alabama,enero,16° / 4°,8 días
...,...,...,...,...,...
13095,Tooele,Utah,agosto,34° / 20°,1 día
13096,Tooele,Utah,septiembre,28° / 14°,2 días
13097,Tooele,Utah,octubre,19° / 7°,2 días
13098,Tooele,Utah,noviembre,11° / 1°,2 días


In [118]:
#check if any county does not have 12 months
month_check = temperature_data.groupby(['county', 'state_name'], as_index=False)['month'].count()
month_check.rename(columns={'month':'month_count'}, inplace=True)

month_check.loc[month_check['month_count'] != 12]

,county,state_name,month_count
133,Chelan,Washington,10
726,Terrebonne,Louisiana,10


In [112]:
#inspect the counties with differences 
temperature_data.loc[temperature_data['county'] == 'Chelan']
temperature_data.loc[temperature_data['county'] == 'Terrebonne']

,county,state_name,month,high/low,rain
4464,Terrebonne,Louisiana,mayo,29° / 20°,junio
4465,Terrebonne,Louisiana,32° / 23°,julio,32° / 24°
4466,Terrebonne,Louisiana,agosto,33° / 24°,enero
4467,Terrebonne,Louisiana,17° / 8°,febrero,19° / 10°
4468,Terrebonne,Louisiana,marzo,22° / 12°,abril
4469,Terrebonne,Louisiana,25° / 16°,mayo,29° / 20°
4470,Terrebonne,Louisiana,junio,32° / 23°,julio
4471,Terrebonne,Louisiana,32° / 24°,agosto,33° / 24°
4472,Terrebonne,Louisiana,septiembre,31° / 22°,octubre
4473,Terrebonne,Louisiana,27° / 16°,noviembre,22° / 11°


In [119]:
#drop data after inspection
drop_rows = temperature_data.loc[temperature_data['county'].isin(['Chelan', 'Terrebonne'])]
temperature_data = temperature_data.drop(drop_rows.index)

In [126]:
#Check if I have all the info for the seleted counties
keys = ['county', 'state_name']
i1 = month_check.set_index(keys).index
i2 = county_selection.set_index(keys).index
missing = county_selection[~i2.isin(i1)]

missing

,Unnamed: 0,county,state_name,state_abbrev


In [121]:
#split temperature in high/low
temp = temperature_data['high/low'].str.split('/', expand=True)
temperature_data['highCelsius'] = temp[0] 
temperature_data['lowCelsius'] = temp[1]

#remove the degree sign from temperature
temperature_data['highCelsius'] = temperature_data.apply(lambda x: x['highCelsius'].strip(), axis=1)
temperature_data['lowCelsius'] = temperature_data.apply(lambda x: x['lowCelsius'].strip(), axis=1)

temperature_data['highCelsius'] = temperature_data.apply(lambda x: x['highCelsius'][:-1], axis=1)
temperature_data['lowCelsius'] = temperature_data.apply(lambda x: x['lowCelsius'][:-1], axis=1)

#remove 'dias' word from rain days
temperature_data['rain'] = temperature_data.apply(lambda x: x['rain'].strip(), axis=1)
temperature_data['rain'] = temperature_data.apply(lambda x: x['rain'][:-5], axis=1)
temperature_data.rename(columns={'rain': "days_of_rain"}, inplace=True)

#remove columns
temperature_data.drop(labels=['high/low'], inplace=True, axis=1)

In [130]:
temperature_data['month'].replace({"enero": "January", "febrero": "February", "marzo": "March", "abril": "April", "mayo": "May", "junio": "June", "julio": "July", "agosto": "August", "setiembre": "September", "octubre": "October", "noviembre": "November", "diciembre": "December"}, inplace=True)

In [132]:
#export data to folder
temperature_data.to_csv("DATA\Rain&Temperaturedata.csv")

In [131]:
temperature_data

,county,state_name,month,days_of_rain,highCelsius,lowCelsius
0,Baldwin,Alabama,May,6,29,17
1,Baldwin,Alabama,June,9,32,21
2,Baldwin,Alabama,July,11,33,22
3,Baldwin,Alabama,August,11,32,22
4,Baldwin,Alabama,January,8,16,4
...,...,...,...,...,...,...
13095,Tooele,Utah,August,,34,20
13096,Tooele,Utah,septiembre,2,28,14
13097,Tooele,Utah,October,2,19,7
13098,Tooele,Utah,November,2,11,1


# County Coordinates Data Gathering

This code gets the latitude and longitude of the location centroids of each county of the USA. These coordinates will be used for visualizations. 

Source: https://en.wikipedia.org/wiki/User:Michael_J/County_table

Data was gathered by using web scraping via BeautifulSoup.

In [2]:
#load HTML contents of webpage to variable using requests and beautifulsoup libraries
url = "https://en.wikipedia.org/wiki/User:Michael_J/County_table"
user_agent =  'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0'
request = urllib.request.Request(url,headers={'User-Agent': user_agent})
page = urllib.request.urlopen(request)
soup = BeautifulSoup(page, 'html.parser')

row2 = soup.find_all("td") #HTML element containing table data

In [47]:
#save data to pandas dataframe
county_coord = pd.DataFrame(columns=['state_abbrev', 'county', 'latitude', 'longitude'])
for i in range(1, len(row2), 14):    
    diction = {'state_abbrev': [row2[i].text], 'county': [row2[i+2].text], 'latitude': [row2[i+11].text], 'longitude': [row2[i+12].text]}
    county_coord = pd.concat([county_coord, pd.DataFrame(diction)], ignore_index=True, sort=False)   

In [48]:
#clean data 
county_coord['longitude'] = county_coord['longitude'].str[:-2]
county_coord['latitude'] = county_coord['latitude'].str[:-1]

In [51]:
county_coord.to_csv("DATA\county_latit&longit_list.csv")